In [1]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import datetime as dt
import numpy as np
import xlwt
%matplotlib inline

In [2]:
conn = pg.connect("dbname='carlplaying' user='drcjar' password='drcjar_is_not_fake'")

cursor = conn.cursor()
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
e = cursor.fetchall()

table_name_list = [i[0] for i in e]

select_template = 'SELECT * FROM {table_name}'
frames_dict = {}
for tname in table_name_list:
    query = select_template.format(table_name = tname)
    frames_dict[tname] = pd.read_sql(query, conn)

In [3]:
# we want a table with participant id, date of recruitment, case or control and centre (can get centre from id)

In [4]:
frames_dict['ipfjes_studyparticipantdetails'].columns

Index(['id', 'created', 'updated', 'consistency_token', 'participant_type',
       'site_ft', 'created_by_id', 'episode_id', 'site_fk_id', 'updated_by_id',
       'comments', 'email_address', 'postal_address', 'want_updates'],
      dtype='object')

In [5]:
frames_dict['ipfjes_demographics'].columns

Index(['id', 'created', 'updated', 'consistency_token', 'hospital_number',
       'nhs_number', 'surname', 'first_name', 'middle_name', 'date_of_birth',
       'religion', 'date_of_death', 'post_code', 'gp_practice_code',
       'death_indicator', 'title_ft', 'marital_status_ft', 'sex_ft',
       'birth_place_ft', 'ethnicity_ft', 'birth_place_fk_id', 'created_by_id',
       'ethnicity_fk_id', 'marital_status_fk_id', 'patient_id', 'sex_fk_id',
       'title_fk_id', 'updated_by_id', 'contact_details', 'phone_number'],
      dtype='object')

In [6]:
pt_lookup = frames_dict['ipfjes_studyparticipantdetails'][['episode_id', 'participant_type']].copy()
pt_lookup.index = pt_lookup['episode_id']
pt_lookup = pt_lookup['participant_type'].to_dict()

In [7]:
df = frames_dict['ipfjes_demographics'].copy()

In [8]:
df['pt'] = df.patient_id.map(lambda x: pt_lookup.get(x))

In [9]:
df = df[['hospital_number', 'pt', 'updated']].copy()

In [10]:
df['centre_id'] = df.hospital_number.str[:2]

In [11]:
df.columns = ['Unique Participant Id', 'Participant Type', 'Activity Date', 'centre_id']

In [12]:
df = df[['centre_id', 'Activity Date', 'Participant Type', 'Unique Participant Id']]

In [13]:
df[df['Participant Type'].isnull()]

,centre_id,Activity Date,Participant Type,Unique Participant Id


In [14]:
df = df[~df['Participant Type'].isnull()] #hack

In [15]:
df.head()

,centre_id,Activity Date,Participant Type,Unique Participant Id
0,02,2017-10-10 13:21:47.570745+01:00,case,020007
1,08,2018-07-05 14:05:28.384524+01:00,case,080016
2,09,2018-05-14 11:29:00.482781+01:00,control,090041
3,01,2017-08-09 12:04:15.256730+01:00,case,010002
4,06,2018-07-12 08:49:38.331069+01:00,case,060003


In [16]:
df = df[df.centre_id != '']

In [17]:
df.centre_id = df.centre_id.astype(int)

In [18]:
lookup = pd.read_csv('trustsites-and-codes.csv')

In [19]:
lookup['centre_id'] = lookup.index + 1

In [20]:
df = pd.merge(df, lookup, on='centre_id')

In [21]:
lookup

,Site Identifier,Site Name,centre_id
0,RR1,HEART OF ENGLAND NHS FOUNDATION TRUST,1
1,7A3C7,MORRISTON HOSPITAL,2
2,RX1,NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST,3
3,RHM01,SOUTHAMPTON GENERAL HOSPITAL,4
4,R0A07,WYTHENSHAWE HOSPITAL,5
5,RGM21,PAPWORTH HOSPITAL NHS FOUNDATION TRUST,6
6,RH801,ROYAL DEVON & EXETER HOSPITAL (WONFORD),7
7,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,8
8,RVJ,NORTH BRISTOL NHS TRUST,9
9,RYJ,IMPERIAL COLLEGE HEALTHCARE NHS TRUST,10


In [22]:
df.centre_id.unique()

array([ 2,  8,  9,  1,  6,  5, 11, 10,  3,  4,  7, 19, 17, 13, 14, 16, 15,
       18, 21, 20, 12])

In [23]:
df = df.sort_values('Activity Date')
df['Activity Date'] = df['Activity Date'].map(lambda x: x.strftime('%d/%m/%Y'))

In [24]:
df['Study Identifier'] = '33390'

In [25]:
df['Study Acronym'] = 'Idiopathic Pulmonary Fibrosis Job Exposures Study (IPF JES)'

In [26]:
df['Participant Type'] = df['Participant Type'].str.replace('case', 'Participant with the relevant condition')

In [27]:
df['Participant Type'] = df['Participant Type'].str.replace('control', 'Participant without condition')

In [28]:
df = df[['Study Identifier', 'Study Acronym', 'Site Identifier', 'Site Name', 'Activity Date', 'Participant Type', 'Unique Participant Id']]


In [29]:
df['Activity Type'] = 'Recruitment'

In [30]:
df

,Study Identifier,Study Acronym,Site Identifier,Site Name,Activity Date,Participant Type,Unique Participant Id,Activity Type
26,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,12/07/2017,Participant with the relevant condition,080010,Recruitment
27,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,12/07/2017,Participant with the relevant condition,080011,Recruitment
342,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,R0A07,WYTHENSHAWE HOSPITAL,14/07/2017,Participant with the relevant condition,050003,Recruitment
416,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,N101H,ABERDEEN ROYAL INFIRMARY,25/07/2017,Participant with the relevant condition,110001,Recruitment
110,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ,NORTH BRISTOL NHS TRUST,25/07/2017,Participant with the relevant condition,090001,Recruitment
111,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ,NORTH BRISTOL NHS TRUST,01/08/2017,Participant with the relevant condition,090002,Recruitment
343,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,R0A07,WYTHENSHAWE HOSPITAL,02/08/2017,Participant with the relevant condition,050007,Recruitment
28,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,04/08/2017,Participant with the relevant condition,080017,Recruitment
174,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RR1,HEART OF ENGLAND NHS FOUNDATION TRUST,08/08/2017,Participant with the relevant condition,010001,Recruitment
173,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RR1,HEART OF ENGLAND NHS FOUNDATION TRUST,09/08/2017,Participant with the relevant condition,010002,Recruitment


In [31]:
df.loc[[3, 7, 11, 25, 29, 40, 43], ['Site Identifier']] = 'RM202' # fixes bug

In [32]:
df.loc[[378], ['Participant Type']]  = 'Participant with the relevant condition'

In [33]:
df['Site Identifier'] = df['Site Identifier'].str.replace('RR1', 'RRK')

In [34]:
df = df.drop_duplicates(subset='Unique Participant Id')

In [35]:
df.to_csv('recruitment.csv')

In [36]:
save_path = '/home/drcjar/Sync/ipfjes-pid/ipfjes-recruitment/'
df.to_csv(save_path+'cpms.csv', index=False)
df.to_excel(save_path+'cpms.xls', index=False)

In [37]:
len(df.drop_duplicates(subset='Unique Participant Id'))

1032

In [38]:
df

,Study Identifier,Study Acronym,Site Identifier,Site Name,Activity Date,Participant Type,Unique Participant Id,Activity Type
26,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,12/07/2017,Participant with the relevant condition,080010,Recruitment
27,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,12/07/2017,Participant with the relevant condition,080011,Recruitment
342,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,R0A07,WYTHENSHAWE HOSPITAL,14/07/2017,Participant with the relevant condition,050003,Recruitment
416,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,N101H,ABERDEEN ROYAL INFIRMARY,25/07/2017,Participant with the relevant condition,110001,Recruitment
110,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ,NORTH BRISTOL NHS TRUST,25/07/2017,Participant with the relevant condition,090001,Recruitment
111,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ,NORTH BRISTOL NHS TRUST,01/08/2017,Participant with the relevant condition,090002,Recruitment
343,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,R0A07,WYTHENSHAWE HOSPITAL,02/08/2017,Participant with the relevant condition,050007,Recruitment
28,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,04/08/2017,Participant with the relevant condition,080017,Recruitment
174,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RRK,HEART OF ENGLAND NHS FOUNDATION TRUST,08/08/2017,Participant with the relevant condition,010001,Recruitment
173,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RRK,HEART OF ENGLAND NHS FOUNDATION TRUST,09/08/2017,Participant with the relevant condition,010002,Recruitment


In [39]:
# df['Unique Participant Id'] = df['Unique Participant Id'].astype('float64')

In [40]:
df1 = pd.read_csv('/home/drcjar/Sync/ipfjes-pid/ipfjes-recruitment/whattheygot1_11_18.csv', dtype={'Unique Participant Id': object})

In [41]:
df1[df1['Unique Participant Id'].duplicated()]

,Study Identifier,Study Acronym,Site Identifier,Site Name,Activity Date,Participant Type,Unique Participant Id,Activity Type
1,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM21,UNIVERSITY HOSPITAL AINTREE,NaN,NaN,NaN,NaN
2,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ20,FRENCHAY HOSPITAL,NaN,NaN,NaN,NaN
3,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RTD01,FREEMAN HOSPITAL,NaN,NaN,NaN,NaN
4,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RBA11,MUSGROVE PARK HOSPITAL,NaN,NaN,NaN,NaN
5,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,NaN,NaN,NaN,NaN
6,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RHU,PORTSMOUTH HOSPITALS NHS TRUST,NaN,NaN,NaN,NaN
7,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RJ122,ST THOMAS' HOSPITAL,NaN,NaN,NaN,NaN


In [42]:
df1['Participant Type'].value_counts()

Participant with the relevant condition    478
Participant without condition              245
Name: Participant Type, dtype: int64

In [43]:
df.head()

,Study Identifier,Study Acronym,Site Identifier,Site Name,Activity Date,Participant Type,Unique Participant Id,Activity Type
26,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,12/07/2017,Participant with the relevant condition,080010,Recruitment
27,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM,AINTREE UNIVERSITY HOSPITAL NHS FOUNDATION TRUST,12/07/2017,Participant with the relevant condition,080011,Recruitment
342,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,R0A07,WYTHENSHAWE HOSPITAL,14/07/2017,Participant with the relevant condition,050003,Recruitment
416,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,N101H,ABERDEEN ROYAL INFIRMARY,25/07/2017,Participant with the relevant condition,110001,Recruitment
110,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ,NORTH BRISTOL NHS TRUST,25/07/2017,Participant with the relevant condition,090001,Recruitment


In [44]:
uploaded = df[df['Unique Participant Id'].isin(df1['Unique Participant Id'])]

In [45]:
toupload = df[~df['Unique Participant Id'].isin(uploaded['Unique Participant Id'])]

In [46]:
toupload.to_excel('toupload.xls', index=False)

In [47]:
df['Participant Type'].value_counts()

Participant with the relevant condition    520
Participant without condition              512
Name: Participant Type, dtype: int64

In [48]:
df1['Participant Type'].value_counts()

Participant with the relevant condition    478
Participant without condition              245
Name: Participant Type, dtype: int64

In [49]:
df1[~df1['Unique Participant Id'].isin(df['Unique Participant Id'])].to_excel('toberemovedplease.xls')

In [50]:
df1[~df1['Unique Participant Id'].isin(df['Unique Participant Id'])]


,Study Identifier,Study Acronym,Site Identifier,Site Name,Activity Date,Participant Type,Unique Participant Id,Activity Type
0,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RGM,ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST,NaN,NaN,NaN,NaN
1,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,REM21,UNIVERSITY HOSPITAL AINTREE,NaN,NaN,NaN,NaN
2,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ20,FRENCHAY HOSPITAL,NaN,NaN,NaN,NaN
3,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RTD01,FREEMAN HOSPITAL,NaN,NaN,NaN,NaN
4,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RBA11,MUSGROVE PARK HOSPITAL,NaN,NaN,NaN,NaN
5,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RTK,ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATIO...,NaN,NaN,NaN,NaN
6,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RHU,PORTSMOUTH HOSPITALS NHS TRUST,NaN,NaN,NaN,NaN
7,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RJ122,ST THOMAS' HOSPITAL,NaN,NaN,NaN,NaN
362,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RRK97,HEARTLANDS HOSPITAL,18/04/2018,Participant without condition,10029,Recruitment
364,33390,Idiopathic Pulmonary Fibrosis Job Exposures St...,RVJ,NORTH BRISTOL NHS TRUST,18/04/2018,Participant without condition,90014,Recruitment


In [51]:
df1[df1['Unique Participant Id'].isin(df['Unique Participant Id'])].to_excel('replacement_datas.xls', index=False)


In [52]:
df1[df1['Unique Participant Id'].isin(df['Unique Participant Id'])]['Participant Type'].value_counts()


Participant with the relevant condition    453
Participant without condition              233
Name: Participant Type, dtype: int64

In [53]:
set(df['Unique Participant Id']) - set(df1[df1['Unique Participant Id'].isin(df['Unique Participant Id'])]['Unique Participant Id'])

{'010075',
 '010076',
 '010077',
 '010078',
 '010079',
 '010080',
 '010081',
 '010082',
 '010083',
 '010084',
 '010085',
 '010086',
 '010087',
 '010088',
 '010089',
 '010090',
 '020015',
 '020016',
 '020017',
 '020018',
 '020019',
 '020020',
 '020021',
 '020022',
 '020023',
 '020024',
 '020025',
 '030063',
 '030064',
 '030065',
 '030066',
 '030067',
 '030068',
 '030069',
 '030070',
 '030071',
 '030072',
 '030073',
 '030074',
 '030075',
 '030076',
 '030077',
 '030078',
 '030079',
 '030080',
 '040076',
 '040077',
 '040078',
 '040079',
 '040080',
 '040081',
 '040082',
 '040083',
 '040084',
 '040085',
 '040086',
 '040087',
 '040088',
 '040089',
 '040090',
 '040091',
 '040092',
 '040093',
 '040094',
 '040095',
 '040096',
 '040097',
 '040098',
 '040099',
 '040100',
 '040101',
 '040102',
 '040103',
 '040104',
 '040105',
 '040106',
 '040107',
 '040108',
 '040109',
 '050064',
 '050065',
 '050066',
 '050067',
 '050068',
 '050069',
 '050070',
 '050071',
 '050072',
 '050073',
 '050074',
 '060045',